In [1]:
import math
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
#import networkx.drawing.layout

Usage: 
1. Generate a network of desired size using the functions below.
2. Output the file to your local drive using "output_network" function. This network can then be used for analysis using "Network_Metrics" script.

In [2]:
#Sparse non-periodic lattice
def generate_sparse_lattice(rows, cols):
    G = nx.Graph()
    
    #Fibonacci words to produce sparseness
    w1 = "010101001010101010010101001010101010010101010100101010010101010100101010010101010100101010101001010100101010101"
    w2 = "10100110101010011010011010101001101010100110100110101010011010011010101001101010100110100110101010011010101001"
    w3 = "01010010010100101001001010010010100101001001010010100100101001001010010100100101001001010010100100101001010010010"
    w4 = "010100101010100101001010101001010101001010010101010010100101010100101010100101001010101001010101001010010101"
    
    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    
    # Add edges
    for r in range(rows):
        for c in range(cols):
            if c < cols - 1 and (not(w1[c] == "1") or not(w2[r] == "1")):
            #if c < cols - 1:
                G.add_edge((r, c), (r, c + 1))  # Horizontal edge
            if r < rows - 1 and (not(w3[c] == "1") or not(w4[r] == "1")):
            #if r < rows - 1:
                G.add_edge((r, c), (r + 1, c))  # Vertical edge
    
    return G


# Sparse periodic lattice
def generate_periodic_sparse_lattice(rows, cols):
    G = nx.Graph()
    
    #Fibonacci words to produce sparseness
    w1 = "010101001010101010010101001010101010010101010100101010010101010100101010010101010100101010101001010100101010101"
    w2 = "10100110101010011010011010101001101010100110100110101010011010011010101001101010100110100110101010011010101001"
    w3 = "01010010010100101001001010010010100101001001010010100100101001001010010100100101001001010010100100101001010010010"
    w4 = "010100101010100101001010101001010101001010010101010010100101010100101010100101001010101001010101001010010101"

    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    
    # Add edges with periodic boundary conditions
    for r in range(rows):
        for c in range(cols):
            if (not(w1[c] == "1") or not(w2[r] == "1")):
                G.add_edge((r, c), (r, (c + 1) % cols))  # Horizontal edge (periodic in columns)
            if (not(w3[c] == "1") or not(w4[r] == "1")):
                G.add_edge((r, c), ((r + 1) % rows, c))  # Vertical edge (periodic in rows)
    
    return G

#Hexagonal non-periodic lattice
def generate_hex_lattice(rows, cols):
    G = nx.Graph()

    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    
    # Add edges
    for r in range(rows):
        for c in range(cols):
            if c < cols - 1:
                G.add_edge((r, c), (r, c + 1))  # Horizontal edge
            if r < rows - 1 and ((c + r) % 2 == 1):
                G.add_edge((r, c), (r + 1, c))  # Vertical edge
    
    return G

# Hexagonal-equivalent lattice
def generate_periodic_hex_lattice(rows, cols):
    G = nx.Graph()
    
    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    
    # Add edges with periodic boundary conditions
    for r in range(rows):
        for c in range(cols):
            G.add_edge((r, c), (r, (c + 1) % cols))  # Horizontal edge (periodic in columns)
            if ((c + r) % 2 == 1):
                G.add_edge((r, c), ((r + 1) % rows, c))  # Vertical edge (periodic in rows)
    
    return G

# Non-periodic Moore grid
def generate_moore_lattice(rows, cols):
    G = nx.Graph()

    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Add edges for Moore neighborhood (8 neighbors)
    for r in range(rows):
        for c in range(cols):
            for dr in [-1, 0, 1]:  # -1, 0, 1 for row
                for dc in [-1, 0, 1]:  # -1, 0, 1 for col
                    if dr == 0 and dc == 0:
                        continue  # Skip the center cell

                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols:  # Check bounds
                        G.add_edge((r, c), (nr, nc))

    return G

# Periodic Moore grid
def generate_periodic_moore_lattice(rows, cols):
    G = nx.Graph()

    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Add edges for periodic Moore neighborhood
    for r in range(rows):
        for c in range(cols):
            for dr in [-1, 0, 1]:  # -1, 0, 1 for row
                for dc in [-1, 0, 1]:  # -1, 0, 1 for col
                    if dr == 0 and dc == 0:
                        continue  # Skip the center cell

                    nr = (r + dr) % rows  # Wrap around rows
                    nc = (c + dc) % cols  # Wrap around cols
                    G.add_edge((r, c), (nr, nc))

    return G

# Large Periodic Moore grid
def generate_periodic_large_lattice(rows, cols):
    G = nx.Graph()

    # Add nodes
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Add edges for periodic Moore neighborhood
    for r in range(rows):
        for c in range(cols):
            # connect to all nodes within range
            for dr in [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]:  
                for dc in [-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]: 
                    if dr == 0 and dc == 0:
                        continue  # Skip the center cell
                    if math.sqrt(dr**2 + dc**2) > 7:
                        continue # Skip if the radius is greater than 7 

                    nr = (r + dr) % rows  # Wrap around rows
                    nc = (c + dc) % cols  # Wrap around cols
                    G.add_edge((r, c), (nr, nc))

    return G


# Non-periodic von Neumann grid (4 neighbors)
def generate_von_neumann_lattice(rows, cols):
    G = nx.grid_graph(dim=(rows,cols),periodic=False)
    return G

# Periodic von Neumann grid
def generate_periodic_von_neumann_lattice(rows, cols):
    G = nx.grid_graph(dim=(rows,cols),periodic=True)
    return G

# Small world network
# size = number of nodes
# degree = average degree
# rewiring = rewiring probability
def generate_small_world_network(size, degree, rewiring):
    G = nx.connected_watts_strogatz_graph(n = size, k=degree, p = rewiring)
    return G

# Preferential attachment graph (Barabasi-Albert)
def generate_preferential_attachment_network(size, degree):
    G = nx.barabasi_albert_graph(n = size, m = degree)
    return G

# Inversing a network creates a line graph of that network. Useful for certain applications.
def inverse_moore_lattice(rows, cols):
    G = generate_moore_lattice(rows, cols)
    G2 = nx.line_graph(G)
    return G2

def inverse_periodic_moore_lattice(rows, cols):
    G = generate_periodic_moore_lattice(rows, cols)
    G2 = nx.line_graph(G)
    return G2

def inverse_von_neumann_lattice(rows, cols):
    G = generate_von_neumann_lattice(rows, cols)
    G2 = nx.line_graph(G)
    return G2

def inverse_periodic_von_neumann_lattice(rows, cols):
    G = generate_periodic_von_neumann_lattice(rows, cols)
    G2 = nx.line_graph(G)
    return G2
    

# Output the network
def output_network(graph, file):
    G = nx.convert_node_labels_to_integers(graph)
    nx.write_gexf(G, file)


Example:

In [3]:
# Generate a small-world network with 1000 nodes, average degree of 20 and 10% rewiring probability
G = generate_small_world_network(1000, 20, 0.1)

# Export the network to the local drive for further analysis
output_network(G, "sample.gexf")